In [1]:
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import sklearn
import os, pathlib, shutil
import kaggle, kagglehub


In [19]:
train_df = pl.read_parquet("train_df.parquet")
test_df = pl.read_parquet("test_df.parquet")
y = train_df.drop_in_place('SalePrice')
X = train_df.drop('Id')

In [21]:
X.shape

(1460, 79)

In [29]:
model = sklearn.ensemble.HistGradientBoostingRegressor(
    loss="squared_error",
    learning_rate=0.1,
    max_iter=100,
    max_leaf_nodes=None,
    max_depth=None,
    max_features=0.8,
    categorical_features="from_dtype",
    min_samples_leaf=10,
    l2_regularization=2.0,
    warm_start=True,
    scoring="neg_root_mean_squared_error",
    validation_fraction=0.2,
    verbose=1,
    random_state=42,
)

In [31]:
model.fit(X,y)
model.score(X,y)


Binning 0.001 GB of training data: 0.028 s
Fitting gradient boosted rounds:
Fit 100 trees in 0.079 s, (10797 total leaves)
Time spent computing histograms: 0.000s
Time spent finding best splits:  0.000s
Time spent applying splits:      0.000s
Time spent predicting:           0.000s


0.9960801644139587

In [32]:
test_df

Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,…,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
i64,cat,cat,f64,i64,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,cat,i64,i64,i64,i64,cat,cat,cat,cat,cat,f64,cat,cat,cat,cat,cat,cat,cat,i64,cat,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,cat,i64,cat,i64,cat,cat,f64,cat,i64,i64,cat,cat,cat,i64,i64,i64,i64,i64,i64,cat,cat,cat,i64,i64,i64,cat,cat,i64
1461,"""20""","""RH""",80.0,11622,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""NAmes""","""Feedr""","""Norm""","""1Fam""","""1Story""",5,6,1961,1961,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""None""",0.0,"""TA""","""TA""","""CBlock""","""TA""","""TA""","""No""","""Rec""",468,"""LwQ""",144,…,0,0,896,0,0,1,0,2,1,"""TA""",5,"""Typ""",0,"""NA""","""Attchd""",1961.0,"""Unf""",1,730,"""TA""","""TA""","""Y""",140,0,0,0,120,0,"""NA""","""MnPrv""","""NA""",0,6,2010,"""WD""","""Normal""",null
1462,"""20""","""RL""",81.0,14267,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Corner""","""Gtl""","""NAmes""","""Norm""","""Norm""","""1Fam""","""1Story""",6,6,1958,1958,"""Hip""","""CompShg""","""Wd Sdng""","""Wd Sdng""","""BrkFace""",108.0,"""TA""","""TA""","""CBlock""","""TA""","""TA""","""No""","""ALQ""",923,"""Unf""",0,…,0,0,1329,0,0,1,1,3,1,"""Gd""",6,"""Typ""",0,"""NA""","""Attchd""",1958.0,"""Unf""",1,312,"""TA""","""TA""","""Y""",393,36,0,0,0,0,"""NA""","""NA""","""Gar2""",12500,6,2010,"""WD""","""Normal""",null
1463,"""60""","""RL""",74.0,13830,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""Gilbert""","""Norm""","""Norm""","""1Fam""","""2Story""",5,5,1997,1998,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""None""",0.0,"""TA""","""TA""","""PConc""","""Gd""","""TA""","""No""","""GLQ""",791,"""Unf""",0,…,701,0,1629,0,0,2,1,3,1,"""TA""",6,"""Typ""",1,"""TA""","""Attchd""",1997.0,"""Fin""",2,482,"""TA""","""TA""","""Y""",212,34,0,0,0,0,"""NA""","""MnPrv""","""NA""",0,3,2010,"""WD""","""Normal""",null
1464,"""60""","""RL""",78.0,9978,"""Pave""","""NA""","""IR1""","""Lvl""","""AllPub""","""Inside""","""Gtl""","""Gilbert""","""Norm""","""Norm""","""1Fam""","""2Story""",6,6,1998,1998,"""Gable""","""CompShg""","""VinylSd""","""VinylSd""","""BrkFace""",20.0,"""TA""","""TA""","""PConc""","""TA""","""TA""","""No""","""GLQ""",602,"""Unf""",0,…,678,0,1604,0,0,2,1,3,1,"""Gd""",7,"""Typ""",1,"""Gd""","""Attchd""",1998.0,"""Fin""",2,470,"""TA""","""TA""","""Y""",360,36,0,0,0,0,"""NA""","""NA""","""NA""",0,6,2010,"""WD""","""Normal""",null
1465,"""120""","""RL""",43.0,5005,"""Pave""","""NA""","""IR1""","""HLS""","""AllPub""","""Inside""","""Gtl""","""StoneBr""","""Norm""","""Norm""","""TwnhsE""","""1Story""",8,5,1992,1992,"""Gable""","""CompShg""","""HdBoard""","""HdBoard""","""None""",0.0,"""Gd""","""TA""","""PConc""","""Gd""","""TA""","""No""","""ALQ""",263,"""Unf""",0,…,0,0,1280,0,0,2,0,2,1,"""Gd""",5,"""Typ""",0,"""NA""","""Attchd""",1992.0,"""RFn""",2,506,"""TA""","""TA""","""Y""",0,82,0,0,144,0,"""NA""","""NA""","""NA""",0,1,2010,"""WD""","""Normal""",null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2915,"""160""","""RM""",21.0,1936,"""Pave""","""NA""","""Reg""","""Lvl""","""AllPub"""

In [34]:
preds = model.predict(test_df.drop('Id', 'SalePrice'))

In [39]:
pl.read_csv('sample_submission.csv')

Id,SalePrice
i64,f64
1461,169277.052498
1462,187758.393989
1463,183583.68357
1464,179317.477511
1465,150730.079977
…,…
2915,167081.220949
2916,164788.778231
2917,219222.4234


In [37]:
submission = pl.DataFrame({'Id': test_df['Id'], 'SalePrice': preds})

In [40]:
submission.write_csv('submission.csv')

In [41]:
!kaggle competitions submit -c home-data-for-ml-course -f submission.csv -m "submission.csv file for home-data-for-ml-course competition"

100%|██████████████████████████████████████| 33.6k/33.6k [00:01<00:00, 23.6kB/s]
Successfully submitted to Housing Prices Competition for Kaggle Learn Users